In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [3]:
data = pd.read_csv('cleaned_data1.csv')
print(data.head())
print(data.info())

   weight  aki_stage  delay_rrt  gender  admission_age  ethnicity  \
0  104.50          1          1       1      66.262081          0   
1  100.00          1          1       1      70.489938          0   
2   98.95          3          0       1      37.700917          0   
3  105.70          3          1       0      59.967156          1   
4   69.00          1          0       1      79.060470          1   

   hematocrit_min  hematocrit_max  hemoglobin_min  hemoglobin_max  ...  \
0            31.4            31.9            11.3            11.3  ...   
1            34.0            34.2            11.3            11.4  ...   
2            27.1            29.4             9.9            10.4  ...   
3            25.0            30.9             8.1            10.1  ...   
4            38.0            38.0            13.2            13.2  ...   

   ptt_max  heart_rate_mean    sbp_mean  dbp_mean   mbp_mean  resp_rate_mean  \
0     47.5        89.625000  132.500000   68.5000  83.090909

In [4]:
# Data preprocessing
X = data.drop('aki_stage', axis=1).values
y = data['aki_stage'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)
y = data['aki_stage'].values - 1

In [5]:
# Separate training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [6]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 3)  # 输出层节点数为类别数，假设为3个类别

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN(input_dim=X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Training model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs.data, 1)
        accuracy = accuracy_score(y_test, predicted)
        f1 = f1_score(y_test, predicted, average='weighted')
        
        # 计算AUC
        probabilities = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_test.cpu().numpy(), probabilities, multi_class='ovr')
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}')

Epoch [1/100], Accuracy: 0.2430, F1 Score: 0.2282, AUC: 0.5202
Epoch [2/100], Accuracy: 0.2654, F1 Score: 0.2693, AUC: 0.5227
Epoch [3/100], Accuracy: 0.3045, F1 Score: 0.3263, AUC: 0.5257
Epoch [4/100], Accuracy: 0.3464, F1 Score: 0.3761, AUC: 0.5279
Epoch [5/100], Accuracy: 0.3966, F1 Score: 0.4341, AUC: 0.5306
Epoch [6/100], Accuracy: 0.4218, F1 Score: 0.4572, AUC: 0.5325
Epoch [7/100], Accuracy: 0.4581, F1 Score: 0.4882, AUC: 0.5339
Epoch [8/100], Accuracy: 0.4944, F1 Score: 0.5179, AUC: 0.5362
Epoch [9/100], Accuracy: 0.5168, F1 Score: 0.5334, AUC: 0.5387
Epoch [10/100], Accuracy: 0.5475, F1 Score: 0.5503, AUC: 0.5395
Epoch [11/100], Accuracy: 0.5810, F1 Score: 0.5725, AUC: 0.5424
Epoch [12/100], Accuracy: 0.6034, F1 Score: 0.5831, AUC: 0.5440
Epoch [13/100], Accuracy: 0.6145, F1 Score: 0.5861, AUC: 0.5457
Epoch [14/100], Accuracy: 0.6117, F1 Score: 0.5797, AUC: 0.5472
Epoch [15/100], Accuracy: 0.6257, F1 Score: 0.5871, AUC: 0.5486
Epoch [16/100], Accuracy: 0.6313, F1 Score: 0.584

In [8]:
# evaluation model
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = accuracy_score(y_test, predicted)
    f1 = f1_score(y_test, predicted, average='weighted')
    
    # 计算最终的AUC
    probabilities = torch.softmax(outputs, dim=1).cpu().numpy()
    auc = roc_auc_score(y_test.cpu().numpy(), probabilities, multi_class='ovr')

    print(f'Final Accuracy: {accuracy:.4f}')
    print(f'Final F1 Score: {f1:.4f}')
    print(f'Final AUC: {auc:.4f}')

Final Accuracy: 0.7095
Final F1 Score: 0.5889
Final AUC: 0.5540
